# 포트폴리오 종목 선정 : 모멘텀 전략
2025.05.18
-> Kospi 상장 종목 중 15개 종목 선정
- 기간 : train의 마지막 1달 (2024-12)

1. train 마지막 2달의 데이터 로드  
    이후 누적수익률(20일) 계산을 위해 2달치 데이터 로드 (2024-11-01 ~ 2024-12-31)

2. 전처리
    - 결측치 제거: 데이터 로드시 결측치 존재 종목 제거
    - 상장 폐지 종목 제거

3. 모멘텀 지표 계산  
    변동성 0 -> 1e6으로 대체  
    리스크 조정 모멘텀 지표 = 누적 수익률 / 변동성  
    수익 대비 변동성(리스크)를 고려한 지표라서 단기 모멘텀 전략에 부합함.

4. 지수 가중평균 계산 (EWMA)  
    최근 데이터에 더 무게를 두어 빠르게 변화하는 시장 상황에 대응 -> 단기 투자에 적합

5. 상위 종목 지표 확인 후 15 종목 선정

In [2]:
# pip install pykrx dart-fss

# library import
import pandas as pd
import numpy as np
import os

from pykrx import stock

import warnings
warnings.filterwarnings('ignore')

## 1. 지표 계산을 위한 데이터 로드
- 기간 : 2024-11-01 ~ 2024-12-31
- pykrx의 KOSPI 상장 종목 종가 데이터
- 휴장일 : 주말, 성탄절, 연말

In [3]:
# Step 1 : 코스피 전체 종목 코드 가져오기
tickers = stock.get_market_ticker_list(market="KOSPI")

# 저장할 데이터프레임 초기화
kospi_close = pd.DataFrame()  # 종가 데이터
kospi_volume = pd.DataFrame()  # 거래량 데이터

# 전체 종목 루프 돌기
for ticker in tickers:
    try:
        # 종목명 가져오기
        name = stock.get_market_ticker_name(ticker)
        print(f"{name} ({ticker}) 다운로드 중...")

        # 해당 종목의 일별 시세 가져오기
        df = stock.get_market_ohlcv_by_date("2024-11-01", "2025-01-01", ticker) # 지표 계산을 위해 12월 30일 전 데이터 필요

        # 종가 및 거래량 추출
        kospi_close[name] = df['종가']
        kospi_volume[name] = df['거래량']
        
    except Exception as e:
        print(f"{ticker} 실패: {e}")

# 인덱스(날짜)를 첫 번째 컬럼으로 만들기
kospi_close.index.name = 'Date'
kospi_volume.index.name = 'Date'

AJ네트웍스 (095570) 다운로드 중...
AK홀딩스 (006840) 다운로드 중...
BGF (027410) 다운로드 중...
BGF리테일 (282330) 다운로드 중...
BNK금융지주 (138930) 다운로드 중...
BYC (001460) 다운로드 중...
BYC우 (001465) 다운로드 중...
CJ (001040) 다운로드 중...
CJ CGV (079160) 다운로드 중...
CJ4우(전환) (00104K) 다운로드 중...
CJ대한통운 (000120) 다운로드 중...
CJ씨푸드 (011150) 다운로드 중...
CJ씨푸드1우 (011155) 다운로드 중...
CJ우 (001045) 다운로드 중...
CJ제일제당 (097950) 다운로드 중...
CJ제일제당 우 (097955) 다운로드 중...
CR홀딩스 (000480) 다운로드 중...
CS홀딩스 (000590) 다운로드 중...
DB (012030) 다운로드 중...
DB손해보험 (005830) 다운로드 중...
DB증권 (016610) 다운로드 중...
DB하이텍 (000990) 다운로드 중...
DH오토넥스 (000300) 다운로드 중...
DI동일 (001530) 다운로드 중...
DKME (015590) 다운로드 중...
DL (000210) 다운로드 중...
DL우 (000215) 다운로드 중...
DL이앤씨 (375500) 다운로드 중...
DL이앤씨2우(전환) (37550L) 다운로드 중...
DL이앤씨우 (37550K) 다운로드 중...
DN오토모티브 (007340) 다운로드 중...
DRB동일 (004840) 다운로드 중...
DSR (155660) 다운로드 중...
DSR제강 (069730) 다운로드 중...
DS단석 (017860) 다운로드 중...
DYP (092780) 다운로드 중...
E1 (017940) 다운로드 중...
ESR켄달스퀘어리츠 (365550) 다운로드 중...
F&F (383220) 다운로드 중...
F&F홀딩스 (007700) 다운로드 중...

In [4]:
display(kospi_close.tail())
print("결측 존재 종목 포함:", kospi_close.shape)

,AJ네트웍스,AK홀딩스,BGF,BGF리테일,BNK금융지주,BYC,BYC우,CJ,CJ CGV,CJ4우(전환),...,효성중공업,효성티앤씨,효성화학,후성,휴니드,휴비스,휴스틸,흥국화재,흥국화재우,흥아해운
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-23,4725,11000,3520,107700,10630,29500,13070,107900,5390,82700,...,424500,239500,44100,5110,7500,2720,3890,3205,5050,1626
2024-12-24,4705,11000,3500,107000,10730,29500,13050,102200,5390,80000,...,393000,245500,42700,5000,7860,2695,3925,3330,6000,1658
2024-12-26,4685,11130,3495,105400,10590,29400,13000,102100,5420,80300,...,389500,241500,42050,4945,7650,2565,3865,3300,5650,1604
2024-12-27,4420,10970,3465,102600,10420,29350,12910,101300,5370,79500,...,386500,233500,40700,4870,7640,2540,3805,3510,6400,1577
2024-12-30,4305,9640,3450,102500,10340,29500,12960,99100,5290,78500,...,393000,239000,39600,4975,7710,2580,3780,3670,8000,1608


결측 존재 종목 포함: (41, 963)


## 2. 전처리
- 결측 종목 제거
- 상장폐지 종목 제거  
    : 상장폐지 종목은 데이터 기간 내에서 일부 날짜만 데이터가 존재하거나, 거래량이 지속적으로 0인 경우
    - 거래량 기준: 특정 기간 동안 거래량이 0인 경우 상장폐지 가능성이 높음
    - 데이터 일관성 기준: 데이터가 일정 기간 이후 존재하지 않는 경우

In [5]:
# Step 2.1: 결측 종목 제거
nan_stocks = kospi_close.columns[kospi_close.isnull().any()]
print("종가 데이터 NaN 존재:\n", nan_stocks)

kospi_close_cleaned = kospi_close.drop(nan_stocks, axis=1)
kospi_volume_cleaned = kospi_volume.drop(nan_stocks, axis=1)
print("결측 존재 종목 제거 후:", kospi_close_cleaned.shape)
print(kospi_close_cleaned.isnull().sum().sum())  # 전체 결측 개수
print(kospi_close_cleaned.isnull().any(axis=1).sum())  # 결측값이 하나라도 있는 날짜 개수

종가 데이터 NaN 존재:
 Index(['GS피앤엘', 'KB발해인프라', 'LG씨엔에스', '달바글로벌', '더본코리아', '서울보증보험', '씨케이솔루션',
       '엠앤씨솔루션'],
      dtype='object')
결측 존재 종목 제거 후: (41, 955)
0
0


In [6]:
# Step 2.2: 상장 폐지 종목 제거

# 거래량 기준으로 상장폐지 후보 검토
low_volume_threshold = kospi_volume_cleaned.index.nunique() // 2  # 영업일의 절반

delisted_candidates = []
for col in kospi_volume_cleaned.columns:
    # 거래량 기준으로 상장폐지 후보 찾기
    low_volume_days = (kospi_volume_cleaned[col] == 0).sum()
    if low_volume_days > low_volume_threshold:
        delisted_candidates.append(col)

# 상장폐지 후보 종목 출력
print("상장폐지 후보 종목:")
print(delisted_candidates)

# 상장폐지 종목 제거
kospi_close_cleaned.drop(delisted_candidates, axis=1, inplace=True)
kospi_volume_cleaned.drop(delisted_candidates, axis=1, inplace=True)

print("\n상장폐지 종목 제거 후 종가 데이터 크기:", kospi_close_cleaned.shape)

상장폐지 후보 종목:
['DH오토넥스', 'DKME', 'IHQ', 'KH 필룩스', '경보제약', '국보', '대동전자', '부산주공', '선도전기', '세원이앤씨', '쌍방울', '에이리츠', '웰바이오텍', '이아이디', '인바이오젠', '주성코퍼레이션', '청호ICT', '카프로', '한창']

상장폐지 종목 제거 후 종가 데이터 크기: (41, 936)


## 3. 날짜별 RAM 계산

In [7]:
# Step 3: 날짜별 RAM 계산
# RAM = cumulative_returns / volatility

# Date 인덱스가 datetime 형식인지 확인
kospi_close_cleaned.index = pd.to_datetime(kospi_close_cleaned.index)

window = 20
ram_df = pd.DataFrame(index=kospi_close_cleaned.index, columns=kospi_close_cleaned.columns)

for col in kospi_close_cleaned.columns:
    for i in range(window - 1, len(kospi_close_cleaned)):
        window_data = kospi_close_cleaned[col].iloc[i - window + 1 : i + 1]
        # NaN 포함되면 건너뛰기
        if window_data.isnull().any():
            continue  # RAM 값 NaN으로 유지(생성 안 함)
        
        cumulative_return = window_data.iloc[-1] / window_data.iloc[0] - 1
        volatility = window_data.pct_change().std()

        # 변동성이 0이면 RAM값 NaN
        # 변동성 0인 부분을 아주 작은 수로 대체
        if volatility == 0 or np.isnan(volatility):
            volatility = 1e-6
            
        ram = cumulative_return / volatility
        ram_df.at[kospi_close_cleaned.index[i], col] = ram

ram_df_filtered = ram_df.loc["2024-12-01":"2024-12-31"]
ram_df_filtered.head()

,AJ네트웍스,AK홀딩스,BGF,BGF리테일,BNK금융지주,BYC,BYC우,CJ,CJ CGV,CJ4우(전환),...,효성중공업,효성티앤씨,효성화학,후성,휴니드,휴비스,휴스틸,흥국화재,흥국화재우,흥아해운
Date,,,,,,,,,,,,,,,,,,,,,
2024-12-02,-1.69954,-7.307037,-3.749065,-4.337845,9.887223,-3.51451,-0.638086,-3.094726,-0.922087,-2.357953,...,-3.587151,-6.282541,-0.833271,-4.978635,1.90479,-0.520825,-1.390847,-3.392618,-3.220562,-6.957208
2024-12-03,-1.485375,-4.320951,0.396392,-2.291496,9.163688,-3.596344,0.947515,-1.288808,1.027801,-0.83913,...,-2.041136,-4.529245,-0.171594,-3.307984,1.178007,0.073344,-3.478006,-1.807267,-1.885042,-4.55229
2024-12-04,-3.182754,-4.517606,-1.68513,-3.923481,7.811108,-3.316667,-0.314485,-2.328275,0.461429,-1.326951,...,-1.566182,-4.433076,-0.689504,-3.244749,-0.849047,-0.789073,-7.261628,-3.407817,-0.50203,-5.930231
2024-12-05,-3.535736,-5.47412,-3.376936,-3.684725,6.813859,-3.842902,-1.096904,-2.266455,-1.23381,-1.855132,...,-1.035448,-4.798522,-1.015976,-4.187919,-0.705955,-1.074843,-7.401347,-3.912182,-1.561724,-7.247249
2024-12-06,-1.138106,-4.710054,0.0,-2.942067,6.7765,-2.571999,0.311951,-0.730472,-0.46605,-0.428984,...,-3.145028,-4.364407,-0.136518,-3.58356,-1.500187,-0.838797,-8.228009,-4.863029,-1.480912,-6.269817


In [8]:
# 모멘텀 지표 결측치 X
print(ram_df_filtered.isnull().sum().sum())  # 전체 결측 개수
print(ram_df_filtered.isnull().any(axis=1).sum())  # 결측값이 하나라도 있는 날짜 개수

0
0


In [9]:
# Step 4: 12월의 RAM 지수가중평균 계산
# alpha 값은 최근 데이터에 대한 가중치 크기
# RAM 지수 같은 금융 데이터에서는 변동성이 크므로 최근 데이터에 더 높은 가중치를 두는 0.4∼0.6을 자주 사용
alpha = 0.4

# ram_df_filtered는 날짜별, 종목별 RAM 값이 담긴 DataFrame
# pandas의 ewm() 함수 활용 (날짜 인덱스 기준)

# 종목별로 지수 가중평균 계산
ewa_ram_df = ram_df_filtered.ewm(alpha=alpha, adjust=False).mean()

# 30일치 전체 지수 가중평균이 나온 상태에서 마지막 날짜 값만 뽑기
final_ewm_ram = ewa_ram_df.iloc[-1]
final_ewm_ram.sort_values(ascending=False).head(20)

파미셀           15.196165
삼양식품          11.572622
이스타코          11.559237
코오롱모빌리티그룹우    11.515028
미원에스씨         11.343420
오리엔트바이오       10.375264
금호전기           9.952640
LS에코에너지        9.441768
HD현대중공업        9.018945
일성건설           9.015732
일진전기           8.206267
DI동일           8.056088
SUN&L          8.032620
흥국화재우          7.642957
대한제강           7.397866
유엔젤            7.194682
두산             7.027263
윌비스            6.997166
씨티알모빌리티        6.962570
다올투자증권         6.912480
Name: 2024-12-30 00:00:00, dtype: float64

## 5. 선정된 주식 종목 EDA
1) 평균거래량 (높으면 유동성이 좋음)

2) 연환산수익률 (높으면 최근 1년간 수익률이 좋음)

3) 변동성: 리스크 크기

4) 샤프비율: 위험 대비 수익성 지표 (높으면 좋음)

In [10]:
# final_ewm_ram에서 상위 15~20개 종목 선택
top_stocks = final_ewm_ram.sort_values(ascending=False).head(20).index.tolist()

# 1) 평균 거래량 (최근 데이터 기준)
avg_volumes = kospi_volume[top_stocks].mean()

# 2) 수익률과 3) 변동성 계산
returns = kospi_close_cleaned[top_stocks].pct_change().dropna()
mean_returns = returns.mean() * 252  # 연환산 수익률
volatility = returns.std() * np.sqrt(252)  # 연환산 변동성

# 4) 샤프 비율 (무위험 수익률 0 가정)
sharpe_ratio = mean_returns / volatility

# DataFrame으로 정리
summary_df = pd.DataFrame({
    '평균거래량': avg_volumes,
    '연환산수익률': mean_returns,
    '연환산변동성': volatility,
    '샤프비율': sharpe_ratio
})
# 해당 종목들의 지수 가중평균 RAM 값 가져오기
ewa_ram_values = final_ewm_ram[top_stocks]
summary_df['지수가중평균RAM'] = ewa_ram_values

summary_df

,평균거래량,연환산수익률,연환산변동성,샤프비율,지수가중평균RAM
파미셀,1.291344e+06,3.428226,0.660963,5.186711,15.196165
삼양식품,9.219373e+04,2.613387,0.602392,4.338352,11.572622
이스타코,3.695332e+06,7.483749,1.840803,4.065481,11.559237
코오롱모빌리티그룹우,1.514683e+05,6.163985,1.712375,3.599671,11.515028
미원에스씨,9.132439e+02,0.390646,0.131413,2.972645,11.343420
오리엔트바이오,6.819402e+06,9.845141,2.089440,4.711857,10.375264
금호전기,3.232404e+05,1.640187,0.538712,3.044646,9.952640
LS에코에너지,2.153328e+05,1.198973,0.615229,1.948824,9.441768
HD현대중공업,3.409216e+05,3.389763,0.674971,5.022090,9.018945
일성건설,6.982253e+06,8.423054,1.930483,4.363185,9.015732


평균거래량 너무 낮은 종목은 실제 매매 시 불리: 미원에스씨, SUN&L

SUN&L: 수익률이 마이너스,샤프비율도 음수

샤프비율이 3~5 이상이면 꽤나 좋음: DI동일 5.54, 파미셀 5.18 등

변동성이 큰 종목(이스타코, 일성건설, 오리엔트바이오)은 수익률도 높지만 리스크가 큼

> - 제거 종목: 미원에스씨(거래량 부족), SUN&L(거래량부족, 수익률음수, 샤프비율 음수)
>  - 우리 프로젝트는 단타+수익성 추구형이므로 변동성이 큰 종목은 가져감.

In [14]:
final_stocks = summary_df.drop(index=['미원에스씨', 'SUN&L'], axis=1).head(15)
final_stocks.to_csv('Selected_Stocks_list.csv', index=True, encoding='cp949')
final_stocks

,평균거래량,연환산수익률,연환산변동성,샤프비율,지수가중평균RAM
파미셀,1.291344e+06,3.428226,0.660963,5.186711,15.196165
삼양식품,9.219373e+04,2.613387,0.602392,4.338352,11.572622
이스타코,3.695332e+06,7.483749,1.840803,4.065481,11.559237
코오롱모빌리티그룹우,1.514683e+05,6.163985,1.712375,3.599671,11.515028
오리엔트바이오,6.819402e+06,9.845141,2.089440,4.711857,10.375264
금호전기,3.232404e+05,1.640187,0.538712,3.044646,9.952640
LS에코에너지,2.153328e+05,1.198973,0.615229,1.948824,9.441768
HD현대중공업,3.409216e+05,3.389763,0.674971,5.022090,9.018945
일성건설,6.982253e+06,8.423054,1.930483,4.363185,9.015732
일진전기,9.686682e+05,0.700887,0.632985,1.107272,8.206267


**연환산수익률(Annualized Return)** 대부분 양호
- 특히 오리엔트바이오(9.85%), 이스타코(7.48%), 일성건설(8.42%)

**샤프비율(Sharpe Ratio)** 대부분 3 이상으로 안정성과 수익률을 같이 고려한 효율적인 포트폴리오.
- 특히 DI동일(5.54), 파미셀(5.19), HD현대중공업(5.02)은 리스크 대비 수익이 좋은 편

**평균 거래량**도 충분한 유동성을 확보한 종목들이 많아 실제 매매하기에도 무리가 적을 것 예상

**변동성**도 적절하게 분포되어 있어 전체적으로 위험 대비 수익이 균형 잡힌 편